In [74]:
import pandas as pd
import numpy as np

In [75]:
temp = pd.read_csv('temperatureaulesETSAB2023_clean.csv')
co2 = pd.read_csv('co2aulesETSAB2023_clean.csv')
hum = pd.read_csv('humidityaulesETSAB2023_clean.csv')

In [76]:
result1 = pd.merge(temp, co2, on=['Date','Aula'])
result1 = result1.drop(columns=['epoch'])
result = pd.merge(result1, hum, on=['Date','Aula'])
result = result.drop(columns=['planta', 'posicio'])
result.head()

,Date,Aula,Temperatura,max_comfort,min_comfort,CO2,outdoors_co2,max_co2,Humidity,min_humidity,max_humidity
0,2023-01-01 00:00:00,A-11,17.80,25,21,492.5,400,1000,55.0,30,60
1,2023-01-01 01:00:00,A-11,17.70,25,21,511.0,400,1000,55.0,30,60
2,2023-01-01 02:00:00,A-11,17.60,25,21,522.0,400,1000,55.0,30,60
3,2023-01-01 03:00:00,A-11,17.50,25,21,564.0,400,1000,54.5,30,60
4,2023-01-01 04:00:00,A-11,17.45,25,21,521.0,400,1000,54.0,30,60


In [77]:
bad = result[(result['Temperatura']>25) & (result['Humidity']>40)]

In [71]:
# DIES FESTIUS:
dies_festius = ['2023-09-25', '2023-10-12', '2023-11-01', '2023-12-06', '2023-12-07', '2023-12-08',
              '2023-12-23','2023-12-24','2023-12-25','2023-12-26','2023-12-27','2023-12-28','2023-12-29',
              '2023-12-30', '2023-12-31','2023-01-01','2023-01-02','2023-01-03','2023-01-04','2023-01-05',
              '2023-01-06','2023-01-07','2023-01-08','2023-01-09','2023-01-10','2023-04-01','2023-04-02',
              '2023-04-03','2023-04-04','2023-04-05','2023-04-06','2023-04-07','2023-04-08','2023-04-09',
              '2023-04-10','2023-05-01','2023-09-01','2023-09-02','2023-09-03','2023-09-04','2023-09-05',
               '2023-09-06','2023-09-07','2023-09-08','2023-09-09','2023-09-10','2023-09-11','2023-09-12']

def marcar_festius_seasonalplot(date):
    return (str(date).split(' ')[0] in dies_festius) or (str(date).split('-')[1] == '08') or (str(date).split('-')[1] == '07')

In [80]:
bad['Date'] = pd.to_datetime(bad['Date'], format='%Y-%m-%d %H:%M:%S')
bad['Month'] = bad['Date'].dt.month
bad['vacation'] = bad['Date'].map(marcar_festius_seasonalplot)
bad = bad[bad['vacation']==False]
bad['day_of_week'] = bad['Date'].dt.weekday
# Create a new column to indicate if the date falls on a weekend (Saturday or Sunday)
bad['is_weekend'] = bad['day_of_week'].isin([5, 6])
bad = bad[bad['is_weekend']==False]
bad['Heat index'] = -8.78469476 + 1.61139411*bad['Temperatura'] + 2.338548839*bad['Humidity'] - 0.14611605*bad['Temperatura']*bad['Humidity'] - 0.012308094*(bad['Temperatura'])**2 - 0.016424828*(bad['Humidity'])**2 + 0.002211732*(bad['Temperatura'])**2*bad['Humidity']+ 0.00072546*bad['Temperatura']*(bad['Humidity'])**2 - 0.000003582*(bad['Temperatura'])**2*(bad['Humidity'])**2
def get_sensation(heat):
    if heat<27:
        return 'High'
    elif heat>=27 and heat<33:
        return 'Caution'
    elif heat>=33 and heat<41:
        return 'Extreme caution'
    elif heat>=41 and heat<54:
        return 'Danger'
    elif heat>=54:
        return 'Extreme danger'
bad['Sensation'] = bad['Heat index'].map(get_sensation)
bad = bad[['Date','Aula','Month','Temperatura','Humidity','Heat index','Sensation']].copy().reset_index()
bad = bad.drop(columns=['index'])
bad.to_csv('highhumtempsensation.csv')
bad.head()

,Date,Aula,Month,Temperatura,Humidity,Heat index,Sensation
0,2023-05-03 12:00:00,A-11,5,25.05,46.0,25.845478,High
1,2023-05-05 12:00:00,A-11,5,25.10,48.5,25.921478,High
2,2023-05-05 13:00:00,A-11,5,25.15,49.0,25.960106,High
3,2023-05-08 10:00:00,A-11,5,25.40,49.0,26.113847,High
4,2023-05-08 11:00:00,A-11,5,26.00,46.0,26.418971,High


In [57]:
bad = bad[['Date','Aula','Month']].copy().reset_index()
bad['Date'] = pd.to_datetime(bad['Date'], format='%Y-%m-%d %H:%M:%S')
bad['Month'] = bad['Date'].dt.month

In [58]:
bad = bad.groupby(['Month', 'Aula']).count().reset_index()
bad = bad.rename(columns={'Date':'count'})
bad.head()

,Month,Aula,count
0,6,A-11,34
1,6,A-12,34
2,6,A-13,37
3,6,A-14,45
4,6,A-21,6


In [59]:
# 13 DE setembre COMENCEN CLASSES hem eliminat vacances i findes
bad.to_csv('highhumtempcount.csv')